In [94]:
import numpy as np
import pandas as pd
import re
import cv2

In [ ]:
!pwd

In [ ]:
import os

folder_path = "./data/gaze/ROI" # sostituire "percorso_della_cartella" con il percorso della cartella che si vuole leggere

all_roi = []
for filename in os.listdir(folder_path):
    path_complete = os.path.join(folder_path, filename)
    if os.path.isfile(path_complete):
        all_roi.append(path_complete)

all_roi




In [ ]:
path1 = "../data/gaze/ROI/roiLook052.asc.npy"
path2 = "../data/gaze/ROI/roiLook002.asc.npy"

pattern = r"roiLook(\d+)\.asc\.npy"

match1 = re.search(pattern, path1)
if match1:
    print(int(match1.group(1)))  # Output: 052

match2 = re.search(pattern, path2)
if match2:
    print(int(match2.group(1)))  # Output: 002




In [ ]:
def extract_sub(path):
    pattern = r"roiLook(\d+)\.asc\.npy"
    return int(re.search(pattern, path).group(1))

In [ ]:
list_subject = [1]

In [ ]:
df_ = pd.DataFrame(columns=['Subject', 'Trial', 'Fixation #', 'X_Coord', 'Y_Coord'])

for path in all_roi:
    data = np.load(path, allow_pickle=True)
    for i in range(len(data)):
        subject = extract_sub(path)
        trial = i+1
        fix_ = data[i]['fixations']
        for num, xy in enumerate(fix_):
            coords = tuple(np.round(xy['mean'], 2))
            row = [int(subject), int(trial), int(num+1), coords[0], coords[1]]
            df_.loc[len(df_)] = row

In [ ]:
len(df_)

In [ ]:
df_.to_csv('../data/gaze/tmp/tmp_fixations.csv', index=False)

In [ ]:
baseline = pd.read_csv('./data/gaze/baselines/18.csv')
baseline

In [ ]:
df_updated = df_.copy(deep=True)
x_cross = 1920/2
y_cross = 1080/2

In [ ]:
all_roi

In [ ]:
fixations = pd.read_csv('./data/gaze/tmp/tmp_fixations.csv')


# for all subjects
for path in all_roi:
    subj = extract_sub(path)
    print("subject: "+str(subj))
    baseline = pd.read_csv('./data/gaze/baselines/'+str(subj)+'.csv')
    print(baseline)
    sub_fixations = fixations[fixations['Subject'] == subj]
    print(sub_fixations)
    break

In [ ]:
baseline

In [ ]:
fixations_updated = fixations.copy(deep=True)
fixations_updated

In [ ]:
df_

# baseline correction, insert missing trials

In [70]:
# for all subjects
for path in all_roi:
    subject = extract_sub(path)
    baseline = pd.read_csv('./data/gaze/baselines/'+str(subject)+'.csv')
    for i in range (1,161):
        current_trial_exist = (len(baseline[baseline['TRIAL_LABEL']==i])!=0)
        if not current_trial_exist:
            baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
    baseline.to_csv('./data/gaze/corrected_baselines/'+str(subject)+'.csv',index=False)

/tmp/ipykernel_118518/4175139398.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
/tmp/ipykernel_118518/4175139398.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
/tmp/ipykernel_118518/4175139398.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  baseline = baseline.append({'RECORDING_SESSION_LABEL':subject,'TRIAL_LABEL':i,'CURRENT_FIX_X':-1,'CURRENT_FIX_Y':-1,'IS_VALID':False},ignore_index=True)
/tmp/ipykernel_1185

# recalculate fixation using baseline

In [71]:
subject=18

In [72]:
baseline = pd.read_csv('./data/gaze/corrected_baselines/'+str(subject)+'.csv')

In [73]:
baseline

,RECORDING_SESSION_LABEL,TRIAL_LABEL,CURRENT_FIX_X,CURRENT_FIX_Y,IS_VALID
0,18,1,977.232099,448.003704,True
1,18,2,938.900000,454.100000,True
2,18,3,875.600000,542.300000,True
3,18,4,938.512274,447.137184,True
4,18,5,930.966917,447.110902,True
...,...,...,...,...,...
155,18,63,-1.000000,-1.000000,False
156,18,65,-1.000000,-1.000000,False
157,18,73,-1.000000,-1.000000,False
158,18,113,-1.000000,-1.000000,False


In [84]:
fixations = pd.read_csv('./data/gaze/tmp/tmp_fixations.csv')
fixations

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,52.0,1.0,1.0,942.04,528.50
1,52.0,1.0,2.0,946.56,520.01
2,52.0,1.0,3.0,1005.51,482.76
3,52.0,1.0,4.0,1061.42,473.28
4,52.0,1.0,5.0,1057.73,474.32
...,...,...,...,...,...
222859,25.0,138.0,28.0,913.57,554.72
222860,25.0,138.0,29.0,919.40,553.31
222861,25.0,138.0,30.0,1082.11,557.33
222862,25.0,138.0,31.0,1075.46,536.27


In [85]:
fixations_updated = fixations.copy(deep=True)

In [86]:
# for all subjects
for path in all_roi:
    subject = extract_sub(path)
    baseline = pd.read_csv('./data/gaze/baselines/'+str(subject)+'.csv')

    for i in range(len(baseline)):
        trial = baseline.TRIAL_LABEL[i]
        valid = baseline.IS_VALID[i]

        if valid:
            x_base = baseline.CURRENT_FIX_X[i]
            y_base = baseline.CURRENT_FIX_Y[i]

        else:
            x_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_X'].mean()
            y_base = baseline[baseline['IS_VALID']==True]['CURRENT_FIX_Y'].mean()

        delta_x, delta_y = (x_cross-x_base), (y_cross-y_base)

        for row in fixations[fixations['Trial'] == trial].iterrows():
            fixations_updated.loc[row[0], 'X_Coord'] = np.round(row[1]['X_Coord'] + delta_x,2)
            fixations_updated.loc[row[0], 'Y_Coord'] = np.round(row[1]['Y_Coord'] + delta_y, 2)


In [89]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,52.0,1.0,1.0,991.74,512.40
1,52.0,1.0,2.0,996.26,503.91
2,52.0,1.0,3.0,1055.21,466.66
3,52.0,1.0,4.0,1111.12,457.18
4,52.0,1.0,5.0,1107.43,458.22
...,...,...,...,...,...
222859,25.0,138.0,28.0,968.47,524.52
222860,25.0,138.0,29.0,974.30,523.11
222861,25.0,138.0,30.0,1137.01,527.13
222862,25.0,138.0,31.0,1130.36,506.07


In [90]:
fixations_updated.to_csv('data/gaze/fixations.csv',index=False)

In [78]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord
0,52.0,1.0,1.0,924.81,620.50
1,52.0,1.0,2.0,929.33,612.01
2,52.0,1.0,3.0,988.28,574.76
3,52.0,1.0,4.0,1044.19,565.28
4,52.0,1.0,5.0,1040.50,566.32
...,...,...,...,...,...
222859,25.0,138.0,28.0,933.97,577.42
222860,25.0,138.0,29.0,939.80,576.01
222861,25.0,138.0,30.0,1102.51,580.03
222862,25.0,138.0,31.0,1095.86,558.97


In [92]:
fixations_updated['ROI'] = np.nan

In [93]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord,ROI
0,52.0,1.0,1.0,991.74,512.40,NaN
1,52.0,1.0,2.0,996.26,503.91,NaN
2,52.0,1.0,3.0,1055.21,466.66,NaN
3,52.0,1.0,4.0,1111.12,457.18,NaN
4,52.0,1.0,5.0,1107.43,458.22,NaN
...,...,...,...,...,...,...
222859,25.0,138.0,28.0,968.47,524.52,NaN
222860,25.0,138.0,29.0,974.30,523.11,NaN
222861,25.0,138.0,30.0,1137.01,527.13,NaN
222862,25.0,138.0,31.0,1130.36,506.07,NaN


In [96]:
mask_12 = cv2.imread("data/gaze/mask/1&2 mask M010.png")
mask_34 = cv2.imread("data/gaze/mask/3&4 mask M057C.png")
mask_56 = cv2.imread("data/gaze/mask/5&6 mask F048C.png")
mask_78 = cv2.imread("data/gaze/mask/7&8 mask F101C.png")

libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: CRC error
libpng warning: iCCP: known incorrect sRGB profile


In [98]:
roi_12 = set()
roi_34 = set()
roi_56 = set()
roi_78 = set()

for y,col in enumerate(mask_12):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_12.add((x+639,y+202))

for y,col in enumerate(mask_34):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_34.add((x+639,y+202))

for y,col in enumerate(mask_56):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_56.add((x+639,y+202))

for y,col in enumerate(mask_78):

    for x,pixel in enumerate(col):

        if 255 not in pixel:

            roi_78.add((x+639,y+202))

# set of coordinates of eye region and mouth nose region

In [99]:
eye_roi = roi_12.union(roi_56)
mouth_nose_roi = roi_34.union(roi_78)

In [100]:
eye_roi

{(968, 540),
 (1042, 489),
 (1044, 535),
 (856, 484),
 (1021, 551),
 (1095, 500),
 (1056, 545),
 (868, 494),
 (835, 546),
 (899, 561),
 (1072, 489),
 (1049, 505),
 (1051, 551),
 (863, 500),
 (898, 494),
 (939, 495),
 (865, 546),
 (840, 489),
 (1015, 490),
 (1079, 505),
 (877, 556),
 (951, 505),
 (893, 500),
 (870, 516),
 (969, 495),
 (886, 460),
 (872, 562),
 (946, 511),
 (847, 505),
 (981, 505),
 (1022, 506),
 (923, 500),
 (1057, 500),
 (900, 516),
 (976, 511),
 (1036, 562),
 (953, 527),
 (1052, 506),
 (854, 521),
 (895, 522),
 (928, 470),
 (1029, 522),
 (930, 516),
 (1004, 465),
 (1045, 466),
 (907, 532),
 (1006, 511),
 (983, 527),
 (1082, 506),
 (999, 471),
 (1059, 522),
 (1034, 465),
 (1061, 568),
 (937, 532),
 (978, 533),
 (1013, 527),
 (1087, 476),
 (901, 471),
 (1066, 538),
 (1008, 533),
 (880, 533),
 (855, 476),
 (915, 527),
 (1020, 543),
 (832, 492),
 (834, 538),
 (908, 487),
 (873, 466),
 (1007, 466),
 (850, 482),
 (910, 533),
 (984, 482),
 (887, 549),
 (885, 476),
 (926, 477

In [101]:
fixations_updated

,Subject,Trial,Fixation #,X_Coord,Y_Coord,ROI
0,52.0,1.0,1.0,991.74,512.40,NaN
1,52.0,1.0,2.0,996.26,503.91,NaN
2,52.0,1.0,3.0,1055.21,466.66,NaN
3,52.0,1.0,4.0,1111.12,457.18,NaN
4,52.0,1.0,5.0,1107.43,458.22,NaN
...,...,...,...,...,...,...
222859,25.0,138.0,28.0,968.47,524.52,NaN
222860,25.0,138.0,29.0,974.30,523.11,NaN
222861,25.0,138.0,30.0,1137.01,527.13,NaN
222862,25.0,138.0,31.0,1130.36,506.07,NaN


In [119]:
for i,row in fixations_updated.iterrows():
    x = int(row['X_Coord'])
    y = int(row['Y_Coord'])
    if (x,y) in eye_roi:
        fixations_updated.loc[i, "ROI"] = "eye"
    if (x,y) in mouth_nose_roi:
        fixations_updated.loc[i, "ROI"] = "mouth_nose"

In [121]:
fixations_updated.to_csv('./data/gaze/fixations_with_ROI.csv',index=False)

In [ ]:
tmp_fixations = pd.read_csv('./data/gaze/tmp/tmp_fixations.csv')
tmp_fixations['Subject']